In [5]:

import matplotlib.pyplot as plt

import torch
from torchvision import datasets, transforms

import numpy as np


#from pytorchcv import plot_results
from torchvision import models
from torch import nn
from torch import optim


ModuleNotFoundError: No module named 'matplotlib'

預訓練

In [9]:
data_dir = 'C:\\Users\\Girl\\Desktop\\1100611\\data'

# TODO: Define transforms for the training data and testing data
train_transforms = transforms.Compose([transforms.RandomRotation(30),
                                       transforms.RandomResizedCrop(224),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.ToTensor()])
#驗證資料集不用增加資料量
test_transforms = transforms.Compose([transforms.Resize(255),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor()])


# Pass transforms in here, then run the next cell to see how the transforms look
train_data = datasets.ImageFolder(data_dir + '/train', transform=train_transforms)
test_data = datasets.ImageFolder(data_dir + '/val', transform=test_transforms)

trainloader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True)
valloader = torch.utils.data.DataLoader(test_data, batch_size=64)
len(trainloader)

157

In [10]:
from torchinfo import summary
import torch.nn.functional as F

class MultiLayerCNN(nn.Module):
  def __init__(self):
    super(MultiLayerCNN, self).__init__()
    self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=2, padding=1)  # 2d卷积层，3通道，32输入通道，卷积核3，步长2，填充值1
    self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=2, padding=1)  # 2d卷积层，32通道，64输入通道，卷积核3，步长2，填充值1
    self.batchnorm1 = nn.BatchNorm2d(64)  # 使用nn.BatchNorm2d归一化

    self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=2, padding=1)  # 2d卷积层，64通道，128输入通道，卷积核3，步长2，填充值1
    self.batchnorm2 = nn.BatchNorm2d(128)

    self.conv4 = nn.Conv2d(128, 256, kernel_size=3, stride=2, padding=1)  # 2d卷积层，128通道，256输入通道，卷积核3，步长2，填充值1
    self.batchnorm3 = nn.BatchNorm2d(256)

    self.maxpool = nn.MaxPool2d(2, 2)  # 2*2最大池化层

    self.fc1 = nn.Linear(256 * 2 * 2, 512)  # 全连接层，输入256*2*2，输出512
    self.dropout = nn.Dropout(0.4)  # 40%的神经元失活
    self.fc2 = nn.Linear(512, 10)  # 全连接层，输入512，输出3
    

  def forward(self, x):
    x = F.relu(self.conv1(x))  # 通过第一个卷积层后使用relu激活函数
    x = F.relu(self.conv2(x))  # 通过第二个卷积层后使用relu激活函数
    x = self.batchnorm1(x)  # 通过第一个归一化
    x = self.maxpool(x)  # 最大池化层
    x = F.relu(self.conv3(x))  # 通过第三个激活函数后使用relu激活函数
    x = self.batchnorm2(x)  # 通过第二个归一化
    x = self.maxpool(x)  # 通过最大池化层
    x = F.relu(self.conv4(x))  # 通过第四个卷积层后使用relu激活函数
    x = self.batchnorm3(x)  # 通过第三个归一化
    x = self.maxpool(x)  # 通过最大池化层
    x = x.view(-1, 256 * 2 * 2)  # 对数据进行reshape，行数自由决定，列数256*2*2
    x = F.relu(self.fc1(x))  # 通过relu激活函数
    x = F.dropout(x)
    x = self.fc2(x)  # 全连接层
    x = F.log_softmax(x, dim=1)  # 选择概率最大的类别
    return x


net = MultiLayerCNN()
print(net)

summary(net, input_size=(64, 3, 224, 224))

MultiLayerCNN(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (batchnorm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (batchnorm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv4): Conv2d(128, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (batchnorm3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (maxpool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=1024, out_features=512, bias=True)
  (dropout): Dropout(p=0.4, inplace=False)
  (fc2): Linear(in_features=512, out_features=10, bias=True)
)


Layer (type:depth-idx)                   Output Shape              Param #
MultiLayerCNN                            [64, 10]                  --
├─Conv2d: 1-1                            [64, 32, 112, 112]        896
├─Conv2d: 1-2                            [64, 64, 56, 56]          18,496
├─BatchNorm2d: 1-3                       [64, 64, 56, 56]          128
├─MaxPool2d: 1-4                         [64, 64, 28, 28]          --
├─Conv2d: 1-5                            [64, 128, 14, 14]         73,856
├─BatchNorm2d: 1-6                       [64, 128, 14, 14]         256
├─MaxPool2d: 1-7                         [64, 128, 7, 7]           --
├─Conv2d: 1-8                            [64, 256, 4, 4]           295,168
├─BatchNorm2d: 1-9                       [64, 256, 4, 4]           512
├─MaxPool2d: 1-10                        [64, 256, 2, 2]           --
├─Linear: 1-11                           [64, 512]                 524,800
├─Linear: 1-12                           [64, 10]              

In [11]:



def validation(model, testloader, criterion):
    accuracy = 0
    test_loss = 0
    for images, labels in testloader:

        #images = images.resize_(x.size(0), 256 * 6 * 6)

        output = model.forward(images)
        test_loss += criterion(output, labels).item()

        ## Calculating the accuracy
        # Model's output is log-softmax, take exponential to get the probabilities
        ps = torch.exp(output)
        top_p, top_class = ps.topk(1, dim=1)
        equals = top_class == labels.view(*top_class.shape)
        accuracy += torch.mean(equals.type(torch.FloatTensor)).item()



    return test_loss, accuracy


def train(model, trainloader, testloader, criterion, optimizer, epochs=100, print_every=157):

    steps = 0
    running_loss = 0
    res = { 'train_loss' : [], 'train_acc': [], 'val_loss': [], 'val_acc': []}
    for e in range(epochs):
        # Model in training mode, dropout is on
        model.train()
        for images, labels in trainloader:
            steps += 1

            # Flatten images into a 784 long vector
            #images.resize_(x.size(0), 256 * 6 * 6)

            optimizer.zero_grad()

            output = model.forward(images)
            loss = criterion(output, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

            if steps % print_every == 0:
                # Model in inference mode, dropout is off
                model.eval()

                # Turn off gradients for validation, will speed up inference
                with torch.no_grad():
                    test_loss, accuracy = validation(model, testloader, criterion)

                print("Epoch: {}/{}.. ".format(e+1, epochs),
                      "Training Loss: {:.3f}.. ".format(running_loss/print_every),
                      "Test Loss: {:.3f}.. ".format(test_loss/len(testloader)),
                      "Test Accuracy: {:.3f}".format(accuracy/len(testloader)))
                res['train_loss'].append(running_loss/print_every)
                res['val_loss'].append(test_loss/len(testloader))
                res['val_acc'].append(accuracy/len(testloader))


                running_loss = 0

                # Make sure dropout and grads are on for training
                model.train()
    return res



In [12]:

'''
input_size = 784
hidden_sizes = [512, 256,64]
output_size = 10

model = nn.Sequential(nn.Linear(input_size, hidden_sizes[0]),
                      nn.ReLU(),
                      nn.Linear(hidden_sizes[0], hidden_sizes[1]),
                      nn.ReLU(),
                      nn.Linear(hidden_sizes[1], hidden_sizes[2]),
                      nn.ReLU(),
                      nn.Linear(hidden_sizes[2], output_size),
                      nn.LogSoftmax(dim=1))
'''
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = models.resnet18(pretrained=True)
model.fc = nn.Linear(512, 10)
model = model.to(device)
criterion = nn.CrossEntropyLoss()
#model=net.to(device)

for param in model.parameters():
    param.requires_grad = True

'''
classifier = nn.Sequential(nn.Linear(1000, 512),
                                 nn.ReLU(),
                                 nn.Dropout(0.2),
                                 nn.Linear(512, 256),
                                 nn.ReLU(),
                                 nn.Dropout(0.4),
                                 nn.Linear(256, 64),
                                 nn.ReLU(),
                                 nn.Dropout(0.4),
                                 nn.Linear(64, 10),
                                 nn.LogSoftmax(dim=1))


model.classifier = classifier
'''


optimizer = optim.Adam(model.parameters(), lr=0.00001)
print(model)

c:\Users\Girl\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Girl\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [13]:
hist = train(model, trainloader, valloader, criterion, optimizer, epochs=3)

In [ ]:
plt.figure(figsize=(15,5))
plt.subplot(121)
plt.plot(hist['train_acc'], label='Training acc')
plt.plot(hist['val_acc'], label='Validation acc')
plt.legend()
plt.subplot(122)
plt.plot(hist['train_loss'], label='Training loss')
plt.plot(hist['val_loss'], label='Validation loss')
plt.legend()


checkpoint = {'input_size': 784,
              'output_size': 10,
              'hidden_layers0':512,
              'hidden_layers1':256,
              'hidden_layers2':64,
              'state_dict': model.state_dict()}



torch.save(model, 'C:/Users/Girl/Desktop/1100611/1100611.pth')

NameError: name 'plt' is not defined

In [ ]:
def view_classify(img, ps, version="MNIST"):
    ''' Function for viewing an image and it's predicted classes.
    '''
    ps = ps.data.numpy().squeeze()

    fig, (ax1, ax2) = plt.subplots(figsize=(6,9), ncols=2)
    ax1.imshow(img.resize_(1, 28, 28).numpy().squeeze())
    ax1.axis('off')
    ax2.barh(np.arange(10), ps)
    ax2.set_aspect(0.1)
    ax2.set_yticks(np.arange(10))
    ax2.set_yticklabels(np.arange(10))
    ax2.set_title('Class Probability')
    ax2.set_xlim(0, 1.1)

    plt.tight_layout()

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)


NameError: name 'torch' is not defined

In [ ]:
import cv2
import os
import csv

model = torch.load('1100611_model.pth')
sample_path = r'./sample_test'
filepath = r'./1100611_2.csv'


#model=load_checkpoint('C:/Users/Girl/Desktop/1100611/110061123.pth')

model.eval()


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

data_dir = sample_path + '/'
filenamelist = os.listdir(data_dir)



pred_data=[]
classes = [
    "0",
    "1",
    "2",
    "3",
    "4",
    "5",
    "6",
    "7",
    "8",
    "9",
  ]

for name in filenamelist:
  img = cv2.imread(data_dir + name)
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  img = cv2.resize(img, (224, 224))
  tran = transforms.ToTensor()
  img = tran(img)
  image_tensor = img.unsqueeze(0)

  
  
  # 取得第0筆資料的圖像和答案
  x= image_tensor
  with torch.no_grad():
    #x = x.resize_(x.size()[0], 784)
    x = x.to(device)
    out = model(x)
    predicted= classes[out[0].argmax(0)]
    pred_data.append(predicted)
    print(f'Predicted: "{predicted}"',out[0].argmax(0))

with open(filepath, 'w',newline='') as csvfile:
  writer = csv.writer(csvfile)
  writer.writerow(['filename', 'class_id'])
  for i in range(len(filenamelist)):
    writer.writerow([filenamelist[i], pred_data[i]])

Predicted: "2" tensor(2)
Predicted: "0" tensor(0)
Predicted: "2" tensor(2)
Predicted: "7" tensor(7)
Predicted: "1" tensor(1)
Predicted: "5" tensor(5)
Predicted: "6" tensor(6)
Predicted: "7" tensor(7)
Predicted: "8" tensor(8)
Predicted: "0" tensor(0)
